Обеспеченность городскими сервисами

In [ ]:
!pip install objectnat folium matplotlib mapclassify

In [ ]:
!pip install momepy

In [ ]:
from objectnat import get_adj_matrix_gdf_to_gdf, get_intermodal_graph
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union
from objectnat import get_service_provision

In [ ]:
import momepy
import networkx as nx
import osmnx as ox

In [ ]:
# Укажи корректные пути к файлам GeoJSON
buildings = ('buildings.geojson')

# Загружаем данные
buildings = gpd.read_file(buildings)
if buildings.crs != "EPSG:3857":
      buildings = buildings.to_crs("EPSG:3857")

In [ ]:
# Укажи корректные пути к файлам GeoJSON
roads = ('roads.geojson')

# Загружаем данные
roads = gpd.read_file(roads)
if roads.crs != "EPSG:3857":
      roads = roads.to_crs("EPSG:3857")

In [ ]:
# Укажи корректные пути к файлам GeoJSON
services = ('service.geojson')

# Загружаем данные
services = gpd.read_file(services)
if services.crs != "EPSG:3857":
      services = services.to_crs("EPSG:3857")

In [ ]:
buildings = buildings[buildings["is_living"] == True].copy() #Оставляем только жилые здания

In [ ]:
buildings ['building_area'] = buildings['building:levels'] * buildings['footprint_area']

In [ ]:
buildings['living_area'] = buildings['building_area'] * 0.7

In [ ]:
buildings['population'] = buildings['living_area']/20 

In [ ]:
buildings["demand"] = buildings["population"] / 1000 * 8 #спрос на сервис ()

In [ ]:
polygon = unary_union([buildings.geometry.to_list() + services.geometry.to_list()]).convex_hull.buffer(
    0.001) #Получаем полигон, опоясывающий геометрии зданий и сервисов

In [ ]:
# Обработка дорог
roads = roads[roads.geom_type.isin(['LineString', 'MultiLineString'])]

GAP_TOLERANCE = 1

def _get_roads(roads):
    """Обработка и объединение дорожной сети"""
    merged = roads.unary_union
    if merged.geom_type == 'MultiLineString':
        roads = gpd.GeoDataFrame(geometry=list(merged.geoms), crs=roads.crs)
    else:
        roads = gpd.GeoDataFrame(geometry=[merged], crs=roads.crs)
    
    roads = roads.explode(index_parts=False).reset_index(drop=True)
    roads.geometry = momepy.close_gaps(roads, GAP_TOLERANCE)
    roads = roads[roads.geom_type.isin(['LineString'])]
    return roads

roads = _get_roads(roads)

In [ ]:
CRS = 32636
SPEED_M_MIN = 1000

def _roads_to_graph(roads):
    """Преобразование дорог в граф"""
    graph = momepy.gdf_to_nx(roads)
    graph.graph['crs'] = roads.crs.to_epsg()
    graph = nx.DiGraph(graph)
    
    for _, _, data in graph.edges(data=True):
        geometry = data['geometry']
        data['time_min'] = geometry.length / SPEED_M_MIN
        
    for n, data in graph.nodes(data=True):
        graph.nodes[n]['x'] = n[0]  # X координата
        graph.nodes[n]['y'] = n[1]  # Y координата

    return graph

roads_G = _roads_to_graph(roads)

In [ ]:
G_intermodal = roads_G

In [ ]:
#G_intermodal = get_intermodal_graph(polygon=polygon, clip_by_bounds=True) #скачиваем граф#

In [ ]:
buildings.to_crs(G_intermodal.graph['crs'], inplace=True)
services.to_crs(G_intermodal.graph['crs'], inplace=True) 

In [ ]:
G_intermodal = nx.convert_node_labels_to_integers(G_intermodal)

In [ ]:
matrix: pd.DataFrame = get_adj_matrix_gdf_to_gdf(gdf_from=buildings,
                                                 gdf_to=services,
                                                 nx_graph=G_intermodal,
                                                 weight="time_min",
                                                 threshold=15
                                                 ) #строим матрицу доступности от сервисов до зданий

In [ ]:
buildings.dropna(subset="demand", inplace=True) #На всякий случай дропаем наны из демандов, мало ли затисались

In [ ]:
matrix.index = matrix.index.astype(int)
services.index = services.index.astype(int)
buildings.index = buildings.index.astype(int)
matrix.columns = matrix.columns.astype(int)
buildings["demand"] = buildings["demand"].astype(int)
services["capacity"] = services["capacity"].astype(int) #Приводим все нужные колонки, индексы и прочее к инту на всякий случай

In [ ]:
build_prov, services_prov, links_prov = get_service_provision(
    buildings=buildings,
    services=services,
    adjacency_matrix=matrix,
    threshold=15,
) #Считаем обеспеченность treshold - нормативная доступность в минутах

In [ ]:
build_prov.explore(column="provison_value") #смотрим что получилось

In [ ]:
build_prov

In [ ]:
import pandas as pd

# Проверяем, что атрибут 'provison_value' существует в GeoDataFrame
if 'provison_value' not in build_prov.columns:
    raise ValueError("GeoDataFrame 'build_prov' must contain a 'provison_value' column.")

# Вычисляем среднее значение и медиану
mean_provision = build_prov['provison_value'].mean()
median_provision = build_prov['provison_value'].median()

# Выводим результаты
print(f"Среднее значение обеспечения: {mean_provision:.2f}")
print(f"Медиана обеспечения: {median_provision:.2f}")


In [ ]:
!pip install folium matplotlib mapclassify

In [ ]:
build_prov.to_file("provision.geojson") #